In [11]:
import requests
from bs4 import BeautifulSoup
from sympy import per
import yfinance as yf
import pandas as pd
import numpy as np

stocks = ['AAPL', 'AMZN', 'TSLA', 'GOOGL', 'JNJ', 'ADBE', 'ZM', 'SQ', 'XOM', 'OXY', 'PG', 'MA', 'V', 'F', 'GE', 'PBF',
          'NEX', 'DIS', 'GME', 'MU', 'SNAP', 'RBLX', 'DDOG']
PFA = []
v = []
score = []
stockslist = []
industry = []
market_cap = []
short_float = []


for stock in stocks:
    try:
        ticker = yf.Ticker(stock)
        current_price = ticker.info['currentPrice']
        ATH = ticker.info['fiftyTwoWeekHigh']
        percent_from_ATH = round((1 - current_price/ATH)*100, 2)
        PFA.append(percent_from_ATH)
        url = f'https://www.alphaquery.com/stock/{stock}/volatility-option-statistics/180-day/historical-volatility'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        volatility = soup.findAll('div', class_ = "indicator-figure-inner")[0]
        vol = float(volatility.text)*100
        vol = round(vol, 2)
        v.append(vol)
        mkcp = ticker.info['marketCap']
        market_cap.append(round(mkcp/1000000000))
        stockslist.append(stock)
        sector = ticker.info['sector']
        industry.append(sector)
        shorts = ticker.info['shortPercentOfFloat']
        shorts = float(shorts*100)
        shorts = round(shorts, 2)
        short_float.append(shorts)
        strength_score = ((0-percent_from_ATH + (vol/2))/2 + (shorts/5))/2
        strength_score = round(strength_score, 2)
        score.append(round(strength_score, 1))
        
    except:
        short_float.append(0)
        continue
    
data = {'Stock': stockslist,
        'Industry': industry,
        'Market Cap (B)': market_cap,
        '% Float Short': short_float,
        'Percent From ATHs': PFA,
        'Volatility': v,
        'Strength Score': score}

df = pd.DataFrame(data)
df = df.sort_values(by='Strength Score', ascending=False)
df.reset_index(drop = True, inplace=True)
df.index = np.arange(1, len(df)+1)
df

#Current date: 2/10/22

,Stock,Industry,Market Cap (B),% Float Short,Percent From ATHs,Volatility,Strength Score
1,PBF,Energy,2,20.75,4.62,74.99,10.3
2,NEX,Energy,2,1.92,1.78,70.98,8.6
3,OXY,Energy,38,5.91,3.57,51.08,6.1
4,DDOG,Technology,55,5.64,11.03,54.88,4.7
5,MU,Technology,103,1.91,6.84,35.79,3.0
6,MA,Financial Services,367,0.73,6.81,33.03,2.5
7,XOM,Energy,331,1.02,5.90,28.04,2.1
8,AAPL,Technology,2825,0.56,5.38,24.96,1.8
9,V,Financial Services,490,1.47,10.49,30.22,1.3
10,GOOGL,Communication Services,1841,0.70,8.06,26.28,1.3
